# Resource planning --- full

In [1]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import logging

logging.basicConfig(level=logging.INFO)

import pickle

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

import cufflinks as cf

cf.go_offline(connected=True)
cf.set_config_file(colorscale="plotly", world_readable=True)

import os
import sys

import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from GPSOdyssey import Kepler
from IPython.display import clear_output
from ipywidgets import fixed, interact, interact_manual, interactive
from keplergl import KeplerGl
from shapely import wkt

# sys.path.insert(0, '../../../mcarptif/')
# sys.path.insert(0, os.path.abspath('../../collection_diagnostics/'))
# sys.path.insert(0, os.path.abspath('../../OSM_processing/'))
import utils.process_gdf as process_gdf

# Extra options
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

from mcarptif.osmnx_network_extract.extract_grptif import NetworkExtract
from mcarptif.osmnx_network_extract.network_code import create_gdf, required_arc_plot
from mcarptif.solver.solve import solve_store_instance
from mcarptif.visualise.route_tables import RouteSummary
from utils.gdf_helpers import create_gdf


def df_style(df):
    if df[0] == "Total":
        return ["font-weight: bold"] * len(df)
    else:
        return [""] * len(df)

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/__init__.py:99: DeprecationWarning:

`np.typeDict` is a deprecated alias for `np.sctypeDict`.



## Load network data

In [3]:
networkh5_file = "../data/05_model_input/sp_files/road_network_simplified_24645.h"
test_network = catalog.load("road_network_simplified_24645_edges")
test_network = test_network.drop(columns=["Unnamed: 0"], errors="ignore")
test_network_directed = catalog.load("road_network_simplified_24645_edges_directed")
nodes = catalog.load("road_network_simplified_24645_nodes")

2022-06-02 22:24:56,338 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_edges` (GeoJSONDataSet)...
2022-06-02 22:24:58,065 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_edges_directed` (GeoJSONDataSet)...
2022-06-02 22:24:59,184 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_nodes` (GeoJSONDataSet)...


In [4]:
test_network.shape

(22189, 19)

In [5]:
%%time
test_network = process_gdf.process_edges(test_network)
test_network.drop_duplicates(["geom_id"], inplace=True)
nodes.sp_index = nodes.index
test_network["arc_index"] = test_network.index
nodes["sp_index"] = nodes.index

CPU times: user 779 ms, sys: 20.3 ms, total: 800 ms
Wall time: 831 ms


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/geopandas/geodataframe.py:199: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



## Run scenarios

In [6]:
%%time
df_lat_lon_match = catalog.load("lat_lon_key_network_match_20220601")
df_producer_geo_all = catalog.load("syn_pop_scenarios_local_20220601")
infrastructure = catalog.load("infrastructure_sample_network_match")
solved = list(catalog.load("resource_summary_table_20220601"))

2022-06-02 22:25:01,006 - kedro.io.data_catalog - INFO - Loading data from `lat_lon_key_network_match_20220601` (CSVDataSet)...
2022-06-02 22:25:01,175 - kedro.io.data_catalog - INFO - Loading data from `syn_pop_scenarios_local_20220601` (PartitionedDataSet)...
2022-06-02 22:25:01,193 - kedro.io.data_catalog - INFO - Loading data from `infrastructure_sample_network_match` (CSVDataSet)...
2022-06-02 22:25:01,197 - kedro.io.data_catalog - INFO - Loading data from `resource_summary_table_20220601` (PartitionedDataSet)...
CPU times: user 163 ms, sys: 26.3 ms, total: 189 ms
Wall time: 196 ms


In [9]:
keys_all = list(df_producer_geo_all)

for keys in keys_all:
    if keys[0] == ".": continue
    if keys in solved: continue
    print(f"\n\n RUNNING SCENARIO {keys}")

    df_producer_geo = df_producer_geo_all[keys]()
    df_producer_geo["lat_lon_id"] = (
        df_producer_geo[["parcelLat", "parcelLon"]]
        .round(8)
        .astype(str)
        .agg("-".join, axis=1)
    )
    df_producer_geo = df_producer_geo.merge(df_lat_lon_match, left_on="lat_lon_id", right_on="lat_lon_id")

    df_producer_geo = df_producer_geo.merge(
        test_network[["arc_id", "geom_id", "geom_id_inv", "geom_id_order", "arc_index"]],
        validate="m:1",
    )
    df_producer_geo["land_use"] = df_producer_geo["mainDwellingType"].fillna("nan")


    df_producer_geo_waste = catalog.load("waste_gen_scenarios_local_20220601")[keys]()
    df_producer_geo_waste = df_producer_geo_waste.loc[df_producer_geo_waste["id"] != 0.01]
    df_producer_geo_waste = df_producer_geo_waste.loc[df_producer_geo_waste["id"].isin(df_producer_geo["id"])]

    routing_parameters = catalog.load(
        "routing_parameters"
    )  # pd.read_excel('../routing_parameters/refuse_parameters.xlsx', sheet_name = 'Refuse_P1')
    routing_parameters.fillna(0, inplace=True)

    calculate = ["bin_service_cost"]
    df_producer_geo.drop(columns=calculate, inplace=True, errors="ignore")
    df_producer_geo["Domestic"] = 1
    df_producer_geo["Non Domestic"] = 0
    df_producer_geo["H/M"] = 0
    df_producer_geo["category"] = "Landed"
    df_producer_geo["land_use"] = "Landed"
    df_producer_geo["total units"] = 1
    df_producer_geo["total_people"] = 1
    df_producer_geo["Postal Code"] = df_producer_geo["id"]

    df_producer_geo = df_producer_geo.merge(routing_parameters)
    df_producer_geo = df_producer_geo.merge(
        df_producer_geo_waste, how="left", validate="1:1"
    )
    df_producer_geo["demand"] = df_producer_geo["waste"]
    # print("\n# offloads (12t)")
    # df_producer_geo.groupby("land_use")["demand"].sum() / 12000  # number of offloads
    # print("\n# units")
    # df_producer_geo.groupby("land_use")["total units"].sum()
    # print("\n# shifts (bin service cost)")
    # df_producer_geo.groupby("land_use")["bin_service_cost"].sum() / (
    #     3600 * 8
    # )  # number of shifts

    key_pois = infrastructure
    key_pois["Postal Code"] = key_pois["id"]

    key_pois["u"] = key_pois["arc_u"]
    key_pois["v"] = key_pois["arc_v"]
    key_pois["key"] = 0
    key_pois["geometry_point"] = key_pois["geometry"]
    key_pois["geometry"] = key_pois["geometry_arc"]
    key_pois = process_gdf.process_edges(key_pois)
    key_pois["geometry"] = key_pois["geometry_point"]
    key_pois = key_pois.merge(test_network[["arc_id", "arc_index"]], validate="1:1")

    offload = key_pois.loc[(key_pois["type"] != "depot")]

    key_depos = key_pois.loc[key_pois["type"] == "depot"]
    key_pois_list = list(key_depos["description"].unique())

    description = "Depot 2"

    key_depot_select = key_depos.copy()
    key_depot_select = key_depot_select.loc[key_depot_select["description"] == description]
    depot_arc_index = key_depot_select.iloc[0]["arc_index"]
    offload = key_pois.loc[key_pois["type"] != "depot"]

    descriptions = ["Offload 2", "Offload 3"]

    key_if_select = offload.copy()
    key_if_select = key_if_select.loc[key_if_select["description"].isin(descriptions)]
    if_arc_index = key_if_select["arc_index"].values

    key_list = list(df_producer_geo["category"].unique())

    descriptions = ["Landed"]

    bad_arcs = [
        "[(18.4571747, -33.9584372), (18.457166, -33.9585368), (18.457156, -33.9585885), (18.4571405, -33.9586324), (18.4571204, -33.9586875), (18.4570842, -33.9587648), (18.4568647, -33.9591331), (18.4568633, -33.9591415), (18.4568634, -33.9591465), (18.4568651, -33.9591518), (18.4568693, -33.9591571), (18.4568826, -33.9591636), (18.4570165, -33.9592207), (18.4570237, -33.9592212), (18.4570302, -33.9592194), (18.4570376, -33.9592149), (18.4570447, -33.9592066), (18.4570542, -33.9591921), (18.457325, -33.958752), (18.4573407, -33.9587173), (18.4573555, -33.9586744), (18.4573665, -33.9586219), (18.4573672, -33.9585957), (18.4573652, -33.958564), (18.4573568, -33.9585404), (18.4573451, -33.9585162), (18.457334, -33.9585017), (18.4573196, -33.9584906), (18.4573008, -33.9584812), (18.457278, -33.9584717), (18.4571747, -33.9584372)]"
    ]

    prod_select = df_producer_geo.copy()
    prod_select = prod_select.loc[~prod_select["geom_id_order"].isin(bad_arcs)]
    prod_select = prod_select.loc[prod_select["category"].isin(descriptions)]

    logging.info("Preparing road network.......\n")
    nodes["u"] = nodes["osmid"]
    network_info = NetworkExtract(
        test_network,
        test_network_directed,
        nodes,
        networkh5_file,
        round_cost=True,
        key_pois=key_pois,
    )
    
    depot = depot_arc_index
    ifs = if_arc_index
    req_arcs = network_info.arc_consolidation_standard(prod_select)
    network_info.load_required_arcs(req_arcs, merge_network=True)
    network_info.set_depot_arc(depot)
    network_info.set_if_arcs(ifs)
    network_info.loc_in_required_arcs()

    network_info.extend_required_inverse_arcs()

    network_info.check_main_list()
    network_info.load_distance_matrix()
    network_info.offload_calculations3D()

    network_info.network_gdf(prod_select)

    logging.info("Done!")

    cut_off = 100000
    bad_arcs = []
    for i in range(len(network_info.d)):
        if network_info.d[i, 0] > cut_off:
            bad_arc = network_info.reqArcList[i]
            bad_links = test_network.loc[test_network["arc_index"] == bad_arc]
            bad_arcs += bad_links["geom_id_order"].tolist()
    assert len(bad_arcs) == 0
    

    cut_off = 100000
    bad_arcs = []
    for i in range(len(network_info.d)):
        if network_info.d[0, i] > cut_off:
            bad_arc = network_info.reqArcList[i]
            bad_links = test_network.loc[test_network["arc_index"] == bad_arc]
            bad_arcs += bad_links["geom_id_order"].tolist()
    assert len(bad_arcs) == 0
    

    d_max = network_info.d.max()
    if d_max == np.inf:
        d_i = network_info.d.argmax(axis=0)[0]
        bad_arc = network_info.reqArcList[d_i]
        print(network_info.d[d_i, :])
        print(network_info.d[:, d_i])
        d_i2 = network_info.d[d_i, :].argmax()
        bad_arc2 = network_info.reqArcList[d_i2]
        print(network_info.d[d_i2, :])
        print(network_info.d[:, d_i2])
        test_network.loc[test_network["arc_index"] == bad_arc]
        test_network.loc[test_network["arc_index"] == bad_arc2]

    scenario_name = "Landed refuse"
    schedule_select = "weekly"
    offload_time = 15 * 60
    max_duration = 10.5 * 3600
    capacity = 10.5 * 1000
    travel_speed = 45 / 3.6
    service_speed = 30 / 3.6

    demand_increase_factor = {"daily": 1, "weekly": 1, "two_one": 1}
    increase_factor = demand_increase_factor[schedule_select]

    logging.info("Uploading scenario inputs...")
    prod_select_update = prod_select.copy()
    prod_arc_update = prod_select_update.copy()

    network_info.producer_demand = prod_arc_update
    prod_arc_update = network_info.add_demand_service_cost_to_ars(
        prod_select_update, service_speed
    )
    _ = pd.DataFrame(
        prod_arc_update.drop(columns=["arc_index"]).sum(numeric_only=True)
    ).astype(int)

    network_info.set_travel_speed(travel_speed)
    network_info.set_service_cost_and_demand(prod_arc_update)
    network_info.set_offload_time(offload_time)
    network_info.check_shapes()

    network_info.update_cost_matrix()
    network_info.update_offload_cost()

    network_info.set_vechile_capacity_constraint(capacity)
    network_info.set_vehicle_duration_constraint(max_duration)
    logging.info("Done")

    solution_df = solve_store_instance(
        "",
        improve=None,
        write_results=False,
        info=network_info,
        overwrite=True,
        test_solution=False,
        full_output=False,
        tollerance=60,
        nnFracLS=1,
        nnFracTS=0.25,
    )
    logging.info("Done generating routes")
    n_route = solution_df["route"].max() + 1
    print(f"Number of routes: {n_route}")

    logging.info("Preparing scenario outputs...")

    network_info.extend_prop_info()
    network_info.add_solution(solution_df, order=False)
    network_info.deconstruct_solution()

    network_info.vehicle_collection_schedule(schedule=schedule_select)
    network_info.extract_wide_offload_totals()

    network_info.add_traversal_time()
    network_info.add_time_formatted()
    network_info.add_constant_duration_time()

    _ = network_info.full_producer_report()

    route_summary = RouteSummary(network_info)
    route_sum_table = route_summary.route_summary()
    logging.info("Done preparing output")

    catalog.save("resource_summary_table_20220601", {keys: route_sum_table})



 RUNNING SCENARIO households_041
2022-06-02 22:27:01,753 - kedro.io.data_catalog - INFO - Loading data from `waste_gen_scenarios_local_20220601` (PartitionedDataSet)...
2022-06-02 22:27:01,772 - kedro.io.data_catalog - INFO - Loading data from `routing_parameters` (ExcelDataSet)...
2022-06-02 22:27:01,976 - root - INFO - Preparing road network.......

2022-06-02 22:27:02,045 - root - INFO - Load required arcs
2022-06-02 22:27:02,046 - root - INFO - Merging with the network
2022-06-02 22:27:02,077 - root - INFO - Set depot
2022-06-02 22:27:02,083 - root - INFO - Set offload facilities
2022-06-02 22:27:02,100 - root - INFO - Extend required arcs with inverse edge arcs


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



2022-06-02 22:27:02,207 - root - INFO - Checking master list
2022-06-02 22:27:02,214 - root - INFO - Load distance matrix: 8143 x 8143


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/leaf.py:544: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



2022-06-02 22:27:08,926 - root - INFO - Creating nearest neighbour lists
2022-06-02 22:27:12,798 - root - INFO - Calculate offloads: 8143 x 8143 x 2
2022-06-02 22:27:22,180 - root - INFO - Done!
2022-06-02 22:27:22,222 - root - INFO - Uploading scenario inputs...
2022-06-02 22:27:22,248 - root - INFO - Setting service cost for network
2022-06-02 22:27:22,262 - root - INFO - Setting demand for network
2022-06-02 22:27:22,276 - root - INFO - Update travel durations.
2022-06-02 22:27:22,566 - root - INFO - Update offload durations.
2022-06-02 22:27:22,972 - root - INFO - Done
2022-06-02 22:27:22,973 - root - INFO - Problem info supplied. Directly proceeding to solve problem.
2022-06-02 22:27:59,878 - root - INFO - Done generating routes
Number of routes: 10
2022-06-02 22:27:59,879 - root - INFO - Preparing scenario outputs...


...

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/leaf.py:544: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tu

2022-06-02 23:31:47,284 - root - INFO - Creating nearest neighbour lists
2022-06-02 23:31:50,610 - root - INFO - Calculate offloads: 8124 x 8124 x 2
2022-06-02 23:31:58,452 - root - INFO - Done!
2022-06-02 23:31:58,494 - root - INFO - Uploading scenario inputs...
2022-06-02 23:31:58,516 - root - INFO - Setting service cost for network
2022-06-02 23:31:58,526 - root - INFO - Setting demand for network
2022-06-02 23:31:58,538 - root - INFO - Update travel durations.
2022-06-02 23:31:58,822 - root - INFO - Update offload durations.
2022-06-02 23:31:59,220 - root - INFO - Done
2022-06-02 23:31:59,221 - root - INFO - Problem info supplied. Directly proceeding to solve problem.
2022-06-02 23:32:32,621 - root - INFO - Done generating routes
Number of routes: 10
2022-06-02 23:32:32,622 - root - INFO - Preparing scenario outputs...


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



2022-06-02 23:32:40,976 - root - INFO - Done preparing output
2022-06-02 23:32:40,977 - kedro.io.data_catalog - INFO - Saving data to `resource_summary_table_20220601` (PartitionedDataSet)...


/Users/ejwillemse/dev/waste_labs_dev/project_rdi_cpt/src/mcarptif/visualise/route_tables.py:146: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

